In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime

In [2]:
def get_soup(url):
    try:
        page = requests.get(url)
        if page.status_code == 200:
            soup = BeautifulSoup(page.text,'html.parser')
            return soup
        else:
            print("page error",page.status_code)
            return None
    except:
        print("Internet error")
        return None

In [3]:
url = 'https://aiweekly.co/'
bsoup = get_soup(url)

In [4]:
# function to find the category of the post
def find_category(class_name):
    in_the_news_1 = bsoup.find('section',class_=''.join(class_name))
    category = in_the_news_1.find('span',class_='category__title__text')
    return category.string


In [33]:
in_the_news_1 = bsoup.find('section',class_='category cc-news')
div_in_section = in_the_news_1()

Ethics


In [5]:
# finding the div which contains all the section of the page
div_body_of_page = bsoup.find('div',class_='issue__body')

# finding all the sections from the div_body_of_page
find_sections = div_body_of_page.find_all('section')

# creating empty list to store the class name of the sections
sec_class_lst = []

# finding section which contains class, extracting the name of the section class and adding to the list
for item in find_sections:
    if item.has_attr('class'):
# sec_class_lst is created in format [['category','name_of_class']] that's why we are extracting index 1 of the list
        sec_class_lst.append(item['class'][1])

# These are unwanted sections form the page that's why we are skipping them
skip_clas_name_lst = ['cc-powered-by','cc-sponsor','cc-sponsorfooter','cc-footer']

# removing the unwanted section class from the list, which sections we don't want
for i in skip_clas_name_lst:
    sec_class_lst.remove(i)

for i in sec_class_lst:
    print(i)


cc-news
cc-inthenews2
cc-robotics
cc-appliedusecases
cc-ethics
cc-research


In [26]:
# function to extract topic name, paragarph and link associated with it

def details(class_name_lst):
    topic=[]
    link = []
    category = []
    para_info = []
    for class_name in class_name_lst:
        section_body = bsoup.find('section',class_=class_name)

        # extracting the achor text and link from each section
        div_in_sec = section_body.find_all('div',class_='item item--issue item--link')
        for div_body in div_in_sec:
            heading3_in_div = div_body.find('h3',class_='item__title')
            link_and_topic = heading3_in_div.find('a')
            topic.append(link_and_topic.string)
            link.append(link_and_topic.get('href'))

            # extracting paragraph from each section
            para_detail = div_body.find('p').text
            para_info.append(para_detail)

            # extracting category form each section
            category.append(find_category(class_name))

        # returning the dictionary of extracted data
    return {'Category':category,
            'Topic':topic,
            'Detail':para_info,
            'Link':link
            }


In [44]:
final_details = details(sec_class_lst)
time_format = datetime.datetime.now()
df = pd.DataFrame(final_details)
df

,Category,Topic,Detail,Link
0,In the News,10 Most Promising Artificial Intelligence Comp...,"Today, the most promising artificial intellige...",https://cur.at/KxibnVB?m=web
1,In The News,The rise of AI could be a great British story....,This investment in widening education and oppo...,https://cur.at/53cZ7kp?m=web
2,In The News,A global law for artificial intelligence?,"At the end of last year, the General Conferenc...",https://cur.at/ITbosDf?m=web
3,In The News,What Humans Lose When We Let AI Decide,Why you should start worrying about artificial...,https://cur.at/nDzEZYy?m=web
4,In The News,Year-End Review & a glimpse into 2022 from Ess...,"As the end of the year approaches, it's time t...",https://cur.at/dlkhf7B?m=web
5,Robotics,Robots and romance: science fiction and science,Do want to sneak in a robot movie to watch on ...,https://cur.at/tr0VOYL?m=web
6,Robotics,Shapeshifting Robots Adapt with Cleverly Desig...,"The first paper, “Shape morphing mechanical me...",https://cur.at/VSnw5cQ?m=web
7,Robotics,"Robots designed to help hospitalized, isolate...",Inside the Barry Art Museum on the campus of O...,https://cur.at/nTnZhZW?m=web
8,Applied use cases,MuZero’s first step from research into the rea...,Collaborating with YouTube to optimise video c...,https://cur.at/y48ZLjp?m=web
9,Applied use cases,Aspinity unveils the first analog machine lear...,Pittsburgh-based Aspinity has unveiled the fir...,https://cur.at/C9i2Vh4?m=web


In [45]:
df.to_excel(str(time_format.date())+'_AI_weekly.xlsx')